#### support request agent

Build and deploy a generic support-request agent with order context and user interaction history.

Outputs include:
- credit recommendation
- refund recommendation
- draft customer response
- past interactions summary
- order details summary

In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")
LLM_MODEL = dbutils.widgets.get("LLM_MODEL")
SUPPORT_AGENT_ENDPOINT_NAME = dbutils.widgets.get("SUPPORT_AGENT_ENDPOINT_NAME")

In [ ]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.support")
spark.sql(f"CREATE TABLE IF NOT EXISTS {CATALOG}.support.support_agent_reports (support_request_id STRING, user_id STRING, order_id STRING, request_text STRING, ts TIMESTAMP, agent_response STRING)")

# Backfill-safe schema evolution for existing tables.
try:
    spark.sql(f"ALTER TABLE {CATALOG}.support.support_agent_reports ADD COLUMNS (request_text STRING)")
except Exception:
    # Column already exists or table metadata already evolved.
    pass

spark.sql(f"""
CREATE OR REPLACE FUNCTION {CATALOG}.ai.get_order_overview(oid STRING)
RETURNS TABLE (
  order_id STRING,
  location STRING,
  items_json STRING,
  customer_address STRING,
  order_created_ts TIMESTAMP
)
RETURN
  SELECT
    order_id,
    location,
    get_json_object(body, '$.items') AS items_json,
    get_json_object(body, '$.customer_addr') AS customer_address,
    try_to_timestamp(ts) AS order_created_ts
  FROM {CATALOG}.lakeflow.all_events
  WHERE order_id = oid AND event_type = 'order_created'
  LIMIT 1
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION {CATALOG}.ai.get_order_timing(oid STRING)
RETURNS TABLE (
  order_id STRING,
  order_created_ts TIMESTAMP,
  delivered_ts TIMESTAMP,
  delivery_duration_minutes FLOAT
)
RETURN
  WITH order_events AS (
    SELECT order_id, event_type, try_to_timestamp(ts) AS event_ts
    FROM {CATALOG}.lakeflow.all_events
    WHERE order_id = oid
  )
  SELECT
    order_id,
    MIN(CASE WHEN event_type='order_created' THEN event_ts END) AS order_created_ts,
    MAX(CASE WHEN event_type='delivered' THEN event_ts END) AS delivered_ts,
    CAST((UNIX_TIMESTAMP(MAX(CASE WHEN event_type='delivered' THEN event_ts END)) - UNIX_TIMESTAMP(MIN(CASE WHEN event_type='order_created' THEN event_ts END))) / 60 AS FLOAT) AS delivery_duration_minutes
  FROM order_events
  GROUP BY order_id
""")

spark.sql(f"""
CREATE OR REPLACE FUNCTION {CATALOG}.ai.get_user_support_history(uid STRING)
RETURNS TABLE (
  support_request_id STRING,
  order_id STRING,
  ts TIMESTAMP,
  agent_response STRING
)
RETURN
  SELECT support_request_id, order_id, ts, agent_response
  FROM {CATALOG}.support.support_agent_reports
  WHERE user_id = uid
  ORDER BY ts DESC
  LIMIT 10
""")

In [ ]:
%pip install -U -qqqq typing_extensions dspy-ai mlflow unitycatalog-openai[databricks] openai databricks-sdk databricks-agents pydantic
%restart_python

In [ ]:
import mlflow
import sys
from databricks.sdk import WorkspaceClient

CATALOG = dbutils.widgets.get("CATALOG")
LLM_MODEL = dbutils.widgets.get("LLM_MODEL")
SUPPORT_AGENT_ENDPOINT_NAME = dbutils.widgets.get("SUPPORT_AGENT_ENDPOINT_NAME")

sys.path.append('../utils')
from uc_state import add

dev_experiment = mlflow.set_experiment(f"/Shared/{CATALOG}_support_agent_dev")
add(CATALOG, "experiments", {"experiment_id": dev_experiment.experiment_id, "name": dev_experiment.name})

In [ ]:
%%writefile agent.py
import json
import warnings
from typing import Literal, Optional
from uuid import uuid4

import dspy
import mlflow
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse
from pydantic import BaseModel, Field
from unitycatalog.ai.core.base import get_uc_function_client

warnings.filterwarnings("ignore", message=".*Ignoring the default notebook Spark session.*")

mlflow.dspy.autolog(log_traces=True)

LLM_MODEL = "{{LLM_MODEL}}"
CATALOG = "{{CATALOG}}"

lm = dspy.LM(f"databricks/{LLM_MODEL}", max_tokens=2000)
dspy.configure(lm=lm)
uc_client = get_uc_function_client()


class SupportReport(BaseModel):
    support_request_id: str
    user_id: str
    order_id: str
    credit_recommendation: Optional[dict] = None
    refund_recommendation: Optional[dict] = None
    draft_response: str
    past_interactions_summary: str
    order_details_summary: str
    decision_confidence: Literal["high", "medium", "low"] = Field(default="medium")
    escalation_flag: bool = Field(default=False)


class SupportTriage(dspy.Signature):
    """Analyze a support request and produce a structured support report.

    Use tools to gather order details and user history before deciding on credit/refund recommendations.
    If there is not enough evidence, avoid monetary recommendations and set escalation_flag=true.
    """

    support_request: str = dspy.InputField(desc="Support request text with support_request_id, user_id, and order_id")
    support_request_id: str = dspy.OutputField()
    user_id: str = dspy.OutputField()
    order_id: str = dspy.OutputField()
    credit_recommendation_json: str = dspy.OutputField(desc="JSON object or null")
    refund_recommendation_json: str = dspy.OutputField(desc="JSON object or null")
    draft_response: str = dspy.OutputField()
    past_interactions_summary: str = dspy.OutputField()
    order_details_summary: str = dspy.OutputField()
    decision_confidence: str = dspy.OutputField(desc="high, medium, low")
    escalation_flag: str = dspy.OutputField(desc="true or false")


def get_order_overview(order_id: str) -> str:
    return str(uc_client.execute_function(f"{CATALOG}.ai.get_order_overview", {"oid": order_id}).value)


def get_order_timing(order_id: str) -> str:
    return str(uc_client.execute_function(f"{CATALOG}.ai.get_order_timing", {"oid": order_id}).value)


def get_user_support_history(user_id: str) -> str:
    return str(uc_client.execute_function(f"{CATALOG}.ai.get_user_support_history", {"uid": user_id}).value)


class SupportModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.react = dspy.ReAct(
            signature=SupportTriage,
            tools=[get_order_overview, get_order_timing, get_user_support_history],
            max_iters=10,
        )

    def forward(self, support_request: str) -> SupportReport:
        result = self.react(support_request=support_request)

        def parse_json_or_none(value: str):
            if value is None:
                return None
            value = str(value).strip()
            if value.lower() in {"", "null", "none"}:
                return None
            try:
                return json.loads(value)
            except Exception:
                return None

        escalation = str(result.escalation_flag).strip().lower() in {"true", "1", "yes"}
        conf = str(result.decision_confidence).strip().lower()
        if conf not in {"high", "medium", "low"}:
            conf = "medium"

        return SupportReport(
            support_request_id=str(result.support_request_id),
            user_id=str(result.user_id),
            order_id=str(result.order_id),
            credit_recommendation=parse_json_or_none(result.credit_recommendation_json),
            refund_recommendation=parse_json_or_none(result.refund_recommendation_json),
            draft_response=str(result.draft_response),
            past_interactions_summary=str(result.past_interactions_summary),
            order_details_summary=str(result.order_details_summary),
            decision_confidence=conf,
            escalation_flag=escalation,
        )


class DSPySupportAgent(ResponsesAgent):
    def __init__(self):
        self.module = SupportModule()

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        user_message = None
        for msg in request.input:
            msg_dict = msg.model_dump() if hasattr(msg, "model_dump") else msg
            if msg_dict.get("role") == "user":
                user_message = msg_dict.get("content", "")
                break
        if not user_message:
            raise ValueError("No user message found")

        result = self.module(support_request=user_message)
        return ResponsesAgentResponse(
            output=[self.create_text_output_item(text=result.model_dump_json(), id=str(uuid4()))],
            custom_outputs=request.custom_inputs,
        )


AGENT = DSPySupportAgent()
mlflow.models.set_model(AGENT)

In [ ]:
import textwrap

agent_source = textwrap.dedent(f'''
import json
import warnings
from typing import Literal, Optional
from uuid import uuid4

import dspy
import mlflow
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import ResponsesAgentRequest, ResponsesAgentResponse
from pydantic import BaseModel, Field
from unitycatalog.ai.core.base import get_uc_function_client

warnings.filterwarnings("ignore", message=".*Ignoring the default notebook Spark session.*")

mlflow.dspy.autolog(log_traces=True)

LLM_MODEL = "{LLM_MODEL}"
CATALOG = "{CATALOG}"

lm = dspy.LM(f"databricks/{{LLM_MODEL}}", max_tokens=2000)
dspy.configure(lm=lm)
uc_client = get_uc_function_client()


class SupportReport(BaseModel):
    support_request_id: str
    user_id: str
    order_id: str
    credit_recommendation: Optional[dict] = None
    refund_recommendation: Optional[dict] = None
    draft_response: str
    past_interactions_summary: str
    order_details_summary: str
    decision_confidence: Literal["high", "medium", "low"] = Field(default="medium")
    escalation_flag: bool = Field(default=False)


class SupportTriage(dspy.Signature):
    """Analyze a support request and produce a structured support report."""

    support_request: str = dspy.InputField(desc="Support request text with support_request_id, user_id, and order_id")
    support_request_id: str = dspy.OutputField()
    user_id: str = dspy.OutputField()
    order_id: str = dspy.OutputField()
    credit_recommendation_json: str = dspy.OutputField(desc="JSON object or null")
    refund_recommendation_json: str = dspy.OutputField(desc="JSON object or null")
    draft_response: str = dspy.OutputField()
    past_interactions_summary: str = dspy.OutputField()
    order_details_summary: str = dspy.OutputField()
    decision_confidence: str = dspy.OutputField(desc="high, medium, low")
    escalation_flag: str = dspy.OutputField(desc="true or false")


def get_order_overview(order_id: str) -> str:
    return str(uc_client.execute_function(f"{{CATALOG}}.ai.get_order_overview", {{"oid": order_id}}).value)


def get_order_timing(order_id: str) -> str:
    return str(uc_client.execute_function(f"{{CATALOG}}.ai.get_order_timing", {{"oid": order_id}}).value)


def get_user_support_history(user_id: str) -> str:
    return str(uc_client.execute_function(f"{{CATALOG}}.ai.get_user_support_history", {{"uid": user_id}}).value)


class SupportModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.react = dspy.ReAct(
            signature=SupportTriage,
            tools=[get_order_overview, get_order_timing, get_user_support_history],
            max_iters=10,
        )

    def forward(self, support_request: str) -> SupportReport:
        result = self.react(support_request=support_request)

        def parse_json_or_none(value: str):
            if value is None:
                return None
            value = str(value).strip()
            if value.lower() in {{"", "null", "none"}}:
                return None
            try:
                return json.loads(value)
            except Exception:
                return None

        escalation = str(result.escalation_flag).strip().lower() in {{"true", "1", "yes"}}
        conf = str(result.decision_confidence).strip().lower()
        if conf not in {{"high", "medium", "low"}}:
            conf = "medium"

        return SupportReport(
            support_request_id=str(result.support_request_id),
            user_id=str(result.user_id),
            order_id=str(result.order_id),
            credit_recommendation=parse_json_or_none(result.credit_recommendation_json),
            refund_recommendation=parse_json_or_none(result.refund_recommendation_json),
            draft_response=str(result.draft_response),
            past_interactions_summary=str(result.past_interactions_summary),
            order_details_summary=str(result.order_details_summary),
            decision_confidence=conf,
            escalation_flag=escalation,
        )


class DSPySupportAgent(ResponsesAgent):
    def __init__(self):
        self.module = SupportModule()

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        user_message = None
        for msg in request.input:
            msg_dict = msg.model_dump() if hasattr(msg, "model_dump") else msg
            if msg_dict.get("role") == "user":
                user_message = msg_dict.get("content", "")
                break
        if not user_message:
            raise ValueError("No user message found")

        result = self.module(support_request=user_message)
        return ResponsesAgentResponse(
            output=[self.create_text_output_item(text=result.model_dump_json(), id=str(uuid4()))],
            custom_outputs=request.custom_inputs,
        )


AGENT = DSPySupportAgent()
mlflow.models.set_model(AGENT)
''')

with open("agent.py", "w", encoding="utf-8") as f:
    f.write(agent_source)

print("Wrote agent.py with concrete CATALOG/LLM_MODEL values")

In [ ]:
import mlflow
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint

sample = spark.sql(f"SELECT support_request_id, user_id, order_id, request_text FROM {CATALOG}.support.raw_support_requests LIMIT 1").collect()
if sample:
    row = sample[0]
    msg = f"support_request_id={row['support_request_id']} user_id={row['user_id']} order_id={row['order_id']} text={row['request_text']}"
else:
    msg = "support_request_id=sample-1 user_id=user-1 order_id=order-1 text=My delivery was late and items were missing."

resources = [DatabricksServingEndpoint(endpoint_name=LLM_MODEL)]
for fn in [f"{CATALOG}.ai.get_order_overview", f"{CATALOG}.ai.get_order_timing", f"{CATALOG}.ai.get_user_support_history"]:
    resources.append(DatabricksFunction(function_name=fn))

input_example = {"input": [{"role": "user", "content": msg}]}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="support_agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
    )

mlflow.set_active_model(model_id=logged_agent_info.model_id)
mlflow.set_registry_uri("databricks-uc")
UC_MODEL_NAME = f"{CATALOG}.ai.support_agent"
uc_registered_model_info = mlflow.register_model(model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME)

In [ ]:
import mlflow
import time
from databricks import agents
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointStateReady

prod_experiment = mlflow.set_experiment(f"/Shared/{CATALOG}_support_agent_prod")
add(CATALOG, "experiments", {"experiment_id": prod_experiment.experiment_id, "name": prod_experiment.name})

deployment_info = agents.deploy(
    model_name=UC_MODEL_NAME,
    model_version=uc_registered_model_info.version,
    scale_to_zero=False,
    endpoint_name=SUPPORT_AGENT_ENDPOINT_NAME,
    environment_vars={"MLFLOW_EXPERIMENT_ID": str(prod_experiment.experiment_id)},
)

workspace = WorkspaceClient()
ready = False
for _ in range(90):
    endpoint = workspace.serving_endpoints.get(name=SUPPORT_AGENT_ENDPOINT_NAME)
    if endpoint.state and endpoint.state.ready == EndpointStateReady.READY:
        ready = True
        break
    time.sleep(10)

if not ready:
    raise RuntimeError(f"Endpoint {SUPPORT_AGENT_ENDPOINT_NAME} did not reach READY within polling window")

add(CATALOG, "endpoints", deployment_info)
print(f"Endpoint {SUPPORT_AGENT_ENDPOINT_NAME} is READY")